In [1]:
from stockfish import Stockfish
import os
from pathlib import Path

import sys
sys.path.append('..')
import chessy.config as cfg

2025-09-13 19:22:17.546 | INFO     | chessy.config:<module>:11 - PROJ_ROOT path is: C:\Users\R coeur\Documents\GitHub\Chessy


In [5]:
LEILA = "../Models/Leila/lc0.exe"

In [3]:
import subprocess
def get_best_move_lc0(fen_position,LCZERO_PATH ):
    # Commande pour exécuter lc0

    # Utilisez subprocess pour lancer lc0 avec les entrées/sorties configurées
    process = subprocess.Popen(LCZERO_PATH, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Envoi de la position FEN et des commandes UCI à lc0
    process.stdin.write("uci\n")                      # Initialisation du mode UCI
    process.stdin.write(f"position fen {fen_position}\n")  # Position de départ
    process.stdin.write("go movetime 1000\n")         # Commande de recherche pour 1 seconde
    process.stdin.flush()

    # Lecture de la sortie pour obtenir le meilleur coup
    best_move = None
    for line in process.stdout:
        print(line.strip())  # Affiche la sortie pour aider au débogage
        if line.startswith("bestmove"):
            best_move = line.split()[1]
            break
    
    # Fermer le processus
    process.stdin.write("quit\n")
    process.stdin.flush()
    process.terminate()
    
    return best_move

# Exemple d'utilisation
fen = "r1bqkbnr/pppppppp/n7/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
best_move = get_best_move_lc0(fen,LEILA)
print(f"Le meilleur coup est : {best_move}")


id name Lc0 v0.31.2
id author The LCZero Authors.
option name WeightsFile type string default <autodiscover>
option name Backend type combo default cuda-auto var cuda-auto var cuda var cuda-fp16 var trivial var random var check var recordreplay var roundrobin var multiplexing var demux
option name BackendOptions type string default
option name Threads type spin default 0 min 0 max 128
option name NNCacheSize type spin default 2000000 min 0 max 999999999
option name MinibatchSize type spin default 0 min 0 max 1024
option name MaxPrefetch type spin default 32 min 0 max 1024
option name CPuct type string default 1.745000
option name CPuctBase type string default 38739.000000
option name CPuctFactor type string default 3.894000
option name TwoFoldDraws type check default true
option name VerboseMoveStats type check default false
option name FpuStrategy type combo default reduction var reduction var absolute
option name FpuValue type string default 0.330000
option name CacheHistoryLength ty

In [6]:
import subprocess
import threading
import queue

class Lc0Engine:
    def __init__(self, lc0_path: str):
        # Lance lc0 une seule fois
        self.process = subprocess.Popen(
            [lc0_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1
        )
        
        # File pour stocker la sortie de lc0
        self.queue = queue.Queue()
        self.thread = threading.Thread(target=self._read_output, daemon=True)
        self.thread.start()

        # Initialisation UCI
        self.send_command("uci")
        self.wait_for("uciok")
        self.send_command("isready")
        self.wait_for("readyok")

    def _read_output(self):
        for line in self.process.stdout:
            self.queue.put(line.strip())

    def send_command(self, command: str):
        self.process.stdin.write(command + "\n")
        self.process.stdin.flush()

    def wait_for(self, keyword: str):
        """Attend qu'une ligne contenant `keyword` apparaisse dans la sortie."""
        while True:
            line = self.queue.get()
            # print("[lc0]", line)  # debug
            if keyword in line:
                return line

    def get_best_move(self, fen: str, movetime=1000):
        """Demande à lc0 le meilleur coup pour une position FEN."""
        self.send_command(f"position fen {fen}")
        self.send_command(f"go movetime {movetime}")
        while True:
            line = self.queue.get()
            # print("[lc0]", line)  # debug
            if line.startswith("bestmove"):
                return line.split()[1]

    def quit(self):
        self.send_command("quit")
        self.process.terminate()


# Exemple d'utilisation :
if __name__ == "__main__":
    engine = Lc0Engine(LEILA)

    fen = "r1bqkbnr/pppppppp/n7/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
    best_move = engine.get_best_move(fen)
    print("Le meilleur coup est :", best_move)

    # Plus tard tu peux redemander d'autres coups avec un autre FEN
    # best_move2 = engine.get_best_move(autre_fen)

    engine.quit()


Le meilleur coup est : e2e4


In [7]:

    best_move = engine.get_best_move(fen)
    print("Le meilleur coup est :", best_move)


engine.quit()

IndentationError: unexpected indent (491411984.py, line 2)

In [18]:
engine = Lc0Engine(LEILA)
autre_fen = "r1bq2r1/b4pk1/p1pp1p2/1p2pP2/1P2P1PB/3P4/1PPQ2P1/R3K2R w"
best_move = engine.get_best_move(autre_fen)
print("Le meilleur coup est :", best_move)
engine.quit()

Le meilleur coup est : g4g5
